# Using Cloud Storage and BigQuery in Python

In [7]:
import pandas as pd
from google.cloud import storage
from google.colab import auth
auth.authenticate_user()

### Download data from Cloud Storage

In [5]:
def download_blob(project_id, bucket_name, source_blob_name, destination_file_name):
    """Downloads a blob from the bucket."""
    # project_id = "your-project-id"
    # bucket_name = "your-bucket-name"
    # source_blob_name = "storage-object-name"
    # destination_file_name = "local/path/to/file"

    storage_client = storage.Client(project_id)

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)

    print(
        "Blob {} downloaded to {}.".format(
            source_blob_name, destination_file_name
        )
    )

In [6]:
download_blob(project_id='ba-780', bucket_name='ba-780', 
              source_blob_name='data/athlete_events.csv', 
              destination_file_name='athlete_events.csv')

Blob data/athlete_events.csv downloaded to athlete_events.csv.


### Loading to pandas and processing

In [9]:
athlete_events = pd.read_csv('athlete_events.csv')
athlete_events.head(3)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN


In [10]:
athlete_events_summer_2012 = athlete_events[athlete_events.Games == '2012 Summer']
athlete_events_summer_2012.shape

(12920, 15)

In [12]:
# write to csv
athlete_events_summer_2012.to_csv('athlete_events_summer_2012.csv')

### Uploading to Cloud Storage bucket

In [11]:
def upload_blob(project_id, bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # project_id = "your-project-id"
    # bucket_name = "your-bucket-name"
    # source_file_name = "local/path/to/file"
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client(project_id)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )

In [13]:
upload_blob(project_id='ba-780', bucket_name='ba-780', 
            source_file_name='athlete_events_summer_2012.csv', 
            destination_blob_name='data/athlete_events_summer_2012.csv')

File athlete_events_summer_2012.csv uploaded to data/athlete_events_summer_2012.csv.


### Download from BigQuery with Pandas

In [16]:
customerChurn = pd.io.gbq.read_gbq(f'''
  SELECT * FROM `ba-780.examples.customerChurn`
''', project_id="ba-780")

In [17]:
customerChurn.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,9732-OUYRN,Female,0,True,False,49,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,False,Credit card (automatic),19.0,918.7,False
1,0661-KQHNK,Female,0,True,True,6,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,False,Credit card (automatic),19.0,105.5,False
2,4709-LKHYG,Female,0,True,True,29,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,False,Electronic check,20.0,540.05,False
3,9824-QCJPK,Male,0,True,False,36,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,False,Mailed check,20.0,666.75,False
4,4716-MRVEN,Female,0,False,False,29,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,False,Mailed check,20.0,599.3,False


In [23]:
customer_churn_monthly_vs_payMethod = customerChurn.groupby('PaymentMethod')['MonthlyCharges'].mean().to_frame().reset_index()
customer_churn_monthly_vs_payMethod

,PaymentMethod,MonthlyCharges
0,Bank transfer (automatic),67.192649
1,Credit card (automatic),66.512385
2,Electronic check,76.255814
3,Mailed check,43.917060


In [25]:
customer_churn_monthly_vs_payMethod.to_gbq('temp.customer_churn_monthly_vs_payMethod',
                                           project_id='ba-780',
                                           if_exists='replace')

1it [00:04,  4.82s/it]
